In [1]:
import cv2
import os
from tkinter import Tk, messagebox

video_path="4.mp4"

### Accelerer la vidéo

In [2]:
def accelerate_video(video_path, original_video_speed, factor=4):
    # Ouvrir la vidéo d'entrée
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Erreur : Impossible de lire la vidéo.")
        return

    # Obtenir les propriétés de la vidéo
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Nouveau FPS (multiplié par le facteur d'accélération)
    new_fps = fps * factor

    # Configurer l'écrivain de vidéo pour la sortie avec le nouveau FPS
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Format de sortie
    out = cv2.VideoWriter(original_video_speed, fourcc, new_fps, (width, height))

    # Lire et écrire chaque frame à une vitesse plus rapide
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)

    # Libérer les ressources
    cap.release()
    out.release()
    print(f"Vidéo accélérée sauvegardée sous : {original_video_speed}")

base_name, ext = os.path.splitext(video_path)
original_video_speed = f"{base_name}_speed{ext}"
accelerate_video(video_path, original_video_speed, factor=4)


Vidéo accélérée sauvegardée sous : 4_speed.mp4


### Clic quand les penalites sont tirés

In [3]:
def record_clicks(input_video, output_file):
    # Ouvrir la vidéo
    cap = cv2.VideoCapture(input_video)
    if not cap.isOpened():
        print("Erreur : Impossible de lire la vidéo.")
        return

    clicks = []
    def click_event(event, x, y, flags, param):
        # Si l'utilisateur clique sur la vidéo
        if event == cv2.EVENT_LBUTTONDOWN:
            time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000  # Temps en secondes
            frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))  # Numéro de la frame
            clicks.append(f"{time:.2f} seconds, frame {frame}")
            print(f"Clic enregistré à {time:.2f} secondes, frame {frame}")

    # Obtenir les dimensions de la vidéo
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Afficher la vidéo et attendre les clics
    cv2.namedWindow("Accéléré", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Accéléré", width, height)
    cv2.setMouseCallback("Accéléré", click_event)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        cv2.imshow("Accéléré", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):  # Appuyer sur 'q' pour quitter
            break

    # Sauvegarder les moments des clics dans un fichier texte
    with open(output_file, 'w') as f:
        for click in clicks:
            f.write(click + "\n")

    # Libérer les ressources
    cap.release()
    cv2.destroyAllWindows()
    print(f"Moments des clics sauvegardés dans : {output_file}")

# Utilisation du code
input_video = original_video_speed  # Vidéo accélérée
output_file = "click_times.txt"  # Fichier pour sauvegarder les clics
record_clicks(input_video, output_file)


Clic enregistré à 9.99 secondes, frame 1200
Clic enregistré à 10.45 secondes, frame 1255
Clic enregistré à 23.69 secondes, frame 2844
Clic enregistré à 28.38 secondes, frame 3406
Moments des clics sauvegardés dans : click_times.txt


### Reconstituer la video avec seulement les videos 

In [4]:
def extract_segments(input_video, click_file, final_accelerate_video, padding=1):
    # Ouvrir la vidéo d'entrée
    cap = cv2.VideoCapture(input_video)
    if not cap.isOpened():
        print("Erreur : Impossible de lire la vidéo.")
        return

    # Obtenir les propriétés de la vidéo d'entrée
    fps_input = int(cap.get(cv2.CAP_PROP_FPS))  # FPS d'origine
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    
    # Ouvrir le fichier de sortie avec le FPS original
    out = cv2.VideoWriter(final_accelerate_video, fourcc, fps_input, (width, height))

    # Lire le fichier de clics
    with open(click_file, 'r') as f:
        lines = f.readlines()

    segments = []
    for line in lines:
        time, frame = line.strip().split(', frame ')
        frame_number = int(frame)
        
        # Calcul des frames avant et après le clic, en utilisant la vidéo d'origine
        start_frame = max(0, frame_number - padding * fps_input)  # Prendre 2 sec avant
        end_frame = min(frame_count, frame_number + padding * fps_input)  # Prendre 2 sec après
        segments.append((start_frame, end_frame))

    # Extraire et écrire chaque segment dans la vidéo de sortie
    for start_frame, end_frame in segments:
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        for frame_num in range(start_frame, end_frame):
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)

    # Libérer les ressources
    cap.release()
    out.release()
    print(f"Vidéo avec les segments des clics sauvegardée sous : {final_accelerate_video}")

# Utilisation du code
input_video = original_video_speed  # Vidéo d'entrée
click_file = "click_times.txt"  # Fichier des clics
final_accelerate_video = "final_video_accelerate.mp4"  # Vidéo finale
extract_segments(input_video, click_file, final_accelerate_video, padding=1)



Vidéo avec les segments des clics sauvegardée sous : final_video_accelerate.mp4


### Deccelerer la vidéo avec les penalités

In [5]:
cap = cv2.VideoCapture('final_video_accelerate.mp4')

fps = cap.get(cv2.CAP_PROP_FPS) 
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  

out = cv2.VideoWriter('final_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps / 4, (width, height))

frame_index = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    out.write(frame)
    frame_index += 1
    if frame_index % 4 != 0: 
        continue

cap.release()
out.release()
cv2.destroyAllWindows()


In [6]:
import cv2
import os

def create_segments(video_path, segment_duration=8, segment_folder="segments"):
    """
    Découpe une vidéo en segments d'une durée spécifiée (en secondes).
    :param video_path: Chemin de la vidéo à découper.
    :param segment_duration: Durée de chaque segment en secondes.
    :param segment_folder: Dossier où enregistrer les segments découpés.
    :return: Liste des chemins des segments créés.
    """
    os.makedirs(segment_folder, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Erreur: Impossible de lire la vidéo.")
        return []

    fps = int(cap.get(cv2.CAP_PROP_FPS))  # Récupérer les FPS de la vidéo
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Récupérer le nombre total de frames
    frames_per_segment = fps * segment_duration  # Durée du segment en nombre de frames
    segment_paths = []

    # Découper la vidéo en segments
    for i in range(0, total_frames, frames_per_segment):
        segment_path = os.path.join(segment_folder, f"segment_{i // frames_per_segment + 1}.mp4")
        segment_paths.append(segment_path)

        # Définir le writer vidéo pour ce segment
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(segment_path, fourcc, fps, 
                              (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

        cap.set(cv2.CAP_PROP_POS_FRAMES, i)  # Définir la position de départ dans la vidéo
        for _ in range(frames_per_segment):
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)

        out.release()

    cap.release()
    print(f"Découpage terminé. {len(segment_paths)} segments créés.")
    return segment_paths

# Exemple d'utilisation :
video_path = 'final_video.mp4'  # Remplacez ce chemin par le chemin de votre vidéo
create_segments(video_path, segment_duration=8)  # Découper la vidéo en segments de 8 secondes


Découpage terminé. 4 segments créés.


['segments\\segment_1.mp4',
 'segments\\segment_2.mp4',
 'segments\\segment_3.mp4',
 'segments\\segment_4.mp4']

In [7]:
import numpy as np

# Fonction pour analyser chaque vidéo
def process_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Erreur: Impossible de lire la vidéo {video_path}.")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # Largeur de la vidéo
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Hauteur de la vidéo
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    print(f"Dimensions de la vidéo {video_path} : {width}x{height}, FPS : {fps}")

    # Initialisation du VideoWriter pour sauvegarder la vidéo annotée
    fourcc = cv2.VideoWriter_fourcc(*'H264')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    prev_frame = None

    while True:
        ret, frame = cap.read()
        if not ret:
            print(f"Fin de la vidéo {video_path} ou erreur de lecture.")
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_frame is not None:
            diff = cv2.absdiff(prev_frame, gray_frame)
            _, thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)

            # Trouver les contours des zones où il y a des différences
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            annotated_frame = frame.copy()
            highest_box = None
            min_y = float('inf')

            # Parcourir les contours et déterminer la boîte avec le plus petit 'y'
            for contour in contours:
                x, y, w, h = cv2.boundingRect(contour)
                if y < min_y:
                    min_y = y
                    highest_box = (x, y, w, h)

            # Si une boîte a été trouvée, dessiner la boîte la plus haute
            if highest_box is not None:
                x, y, w, h = highest_box
                cv2.rectangle(annotated_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(annotated_frame, f"x:{x}, y:{y}", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                print(f"Coordonnées de la boîte la plus haute : x={x}, y={y}, w={w}, h={h}")

            out.write(annotated_frame)

        prev_frame = gray_frame
        frame_count += 1

    cap.release()
    out.release()

    print(f"Vidéo annotée sauvegardée avec succès : {output_path}")

# Analyser toutes les vidéos dans le dossier 'segments/'
segment_folder = 'segments/'
segment_files = [f for f in os.listdir(segment_folder) if f.endswith('.mp4')]

# Créer un dossier pour les vidéos annotées
os.makedirs("segments_annotated", exist_ok=True)

# Traiter chaque vidéo dans le dossier 'segments/'
for segment_file in segment_files:
    video_path = os.path.join(segment_folder, segment_file)
    output_path = os.path.join("segments_annotated", f"annotated_{segment_file}")
    
    # Analyser et annoter la vidéo
    process_video(video_path, output_path)

cv2.destroyAllWindows()
print("Traitement terminé pour toutes les vidéos.")


Dimensions de la vidéo segments/segment_1.mp4 : 1920x1080, FPS : 30
Coordonnées de la boîte la plus haute : x=1384, y=224, w=1, h=1
Coordonnées de la boîte la plus haute : x=548, y=451, w=1, h=2
Coordonnées de la boîte la plus haute : x=1914, y=372, w=4, h=1
Coordonnées de la boîte la plus haute : x=0, y=333, w=9, h=10
Coordonnées de la boîte la plus haute : x=13, y=317, w=8, h=10
Coordonnées de la boîte la plus haute : x=1851, y=182, w=1, h=5
Coordonnées de la boîte la plus haute : x=1634, y=164, w=1, h=1
Coordonnées de la boîte la plus haute : x=1771, y=195, w=1, h=1
Coordonnées de la boîte la plus haute : x=64, y=265, w=8, h=7
Coordonnées de la boîte la plus haute : x=81, y=254, w=3, h=5
Coordonnées de la boîte la plus haute : x=89, y=243, w=7, h=9
Coordonnées de la boîte la plus haute : x=99, y=234, w=7, h=6
Coordonnées de la boîte la plus haute : x=111, y=224, w=9, h=6
Coordonnées de la boîte la plus haute : x=1763, y=206, w=1, h=2
Coordonnées de la boîte la plus haute : x=134, y=

In [14]:
import cv2
import os
from tkinter import Tk, messagebox

# Variables globales
penalty_positions = []  # Liste pour stocker les positions et frames des pénalités
frame_index = 0  # Indice de la frame actuelle
global frame  # Déclarer `frame` comme variable globale pour pouvoir la modifier dans `click_event`

frame = None  # Variable pour stocker la frame actuelle
lines = []

# Fonction de gestion des clics
def click_event(event, x, y, flags, param):
    """
    Fonction pour gérer les clics de souris.
    """
    global penalty_positions, frame_index, frame
    if event == cv2.EVENT_LBUTTONDOWN:  # Clic gauche
        penalty_positions.append((frame_index, (x, y)))  # Ajouter la frame et la position du clic
        print(f"Pénalité détectée à la frame {frame_index}, position : x={x}, y={y}")
        # Dessiner un point rouge sur la frame actuelle
        cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)  # Dessiner un point rouge

# Charger les segments vidéo
segment_folder = "segments"
segment_files = sorted([f for f in os.listdir(segment_folder) if f.endswith('.mp4')])

# Lire ou créer le fichier penalty_positions.txt
positions_file = 'penalty_positions.txt'
if not os.path.exists(positions_file):
    # Si le fichier n'existe pas, on le crée
    open(positions_file, 'w').close()

# Fonction pour poser la question et obtenir la réponse
def ask_question(segment_index):
    root = Tk()
    root.withdraw()  # Cacher la fenêtre principale de Tkinter

    response = messagebox.askquestion(
        "Question",
        f"La balle est-elle rentrée pour le segment {segment_index + 1} ?",
        icon='question'
    )
    root.destroy()  # Détruire la fenêtre après avoir obtenu la réponse
    return "Oui" if response == "yes" else "Non"

# Processus pour chaque segment
for index, segment_file in enumerate(segment_files):
    # Charger le segment vidéo
    segment_path = os.path.join(segment_folder, segment_file)
    cap = cv2.VideoCapture(segment_path)

    if not cap.isOpened():
        print(f"Erreur: Impossible de lire le segment {segment_file}")
        continue

    # Obtenir les dimensions et le FPS de la vidéo
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    delay = int(700 / fps)  # Délai en millisecondes pour chaque frame

    # Redimensionner la fenêtre pour correspondre à la taille de la vidéo
    cv2.namedWindow("Segment", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Segment", width, height)

    # Lire et afficher le segment à la vitesse correcte
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_index = int(cap.get(cv2.CAP_PROP_POS_FRAMES))  # Récupérer l'index de la frame actuelle
        cv2.imshow("Segment", frame)

        cv2.setMouseCallback("Segment", click_event)  # Associer la gestion des clics

        if cv2.waitKey(delay) & 0xFF == ord('q'):  # Respecter la vitesse de la vidéo
            break

    cap.release()

    # Demander la réponse après avoir visionné la vidéo
    reponse = ask_question(index)

    # Ajouter les positions de pénalités et les réponses à la liste
    for frame_num, position in penalty_positions:
        lines.append(f"Video: {segment_file}, Frame: {frame_num}, Position: {position}, Rentree : {reponse}\n")

    # Réinitialiser la liste des positions après chaque segment
    penalty_positions.clear()

    # Fermer toutes les fenêtres après avoir visionné chaque segment
    cv2.destroyAllWindows()

# Écrire les lignes mises à jour dans le fichier
with open(positions_file, 'w') as file:
    file.writelines(lines)

print("Mises à jour enregistrées dans penalty_positions.txt.")


Pénalité détectée à la frame 74, position : x=852, y=537
Pénalité détectée à la frame 115, position : x=1329, y=559
Pénalité détectée à la frame 111, position : x=1608, y=514
Pénalité détectée à la frame 61, position : x=1327, y=537
Mises à jour enregistrées dans penalty_positions.txt.
